# Introduction to the problem

For this question, you need to have a conversation with the interviewer to understand what type of vehicles it can support and whether the parking lot has multiple levels and so on. Parking can be free or it can be charged. It can be charged in a number of different ways (it can be free for the first hour, paid for the next few). Charges can also be different depending on how big the vehicle is, etc. 

Q. How many floors are going to be there in this parking lot? 

Q. What kinds of vehicles will be parked? Will their sizes differ and do I need to account for that in the parking lot? Do I need to have designated spots for that in the parking lot? 

Q. Is there a payment sort of system where the driver is assigned a parking spot? 

For this exercise, let's assume the following. 

A. There are multiple floors 
B. Different types of vehicles exist that can be parked. (Cars, Limos and Semi Trucks). Car will take up 1 parking spot, Limos 2 and Semi Trucks 3. The vehicles need to take up consecutive parking spots.  

C. There will be a payment system that will assign a spot to the driver and will charge them once they leave the parking spot. The payment system will assign a spot based on the lowest spot available in the parking garage. 

In [45]:
from typing import Optional

In [46]:
class Vehicle:
    def __init__(self, spotSize):
        self.spotSize = spotSize 

    def getSpotSize(self):
        return self.spotSize 

In [47]:
class Driver: 
    def __init__(self, idNumber, vehicle):
        self.id = idNumber 
        self.vehicle = vehicle 
        self.paymentDue = 0 

    def getVehicle(self):
        return self.vehicle 

    def getId(self):
        return self.id 

    def charge(self, amount):
        self.paymentDue += amount 

In [48]:
class Car(Vehicle): 
    def __init__(self):
        super().__init__(1)
        
class Limo(Vehicle):
    def __init__(self):
        super().__init__(2)

class Truck(Vehicle):
    def __init__(self):
        super().__init__(3)

In [49]:
class ParkingFloor: 
    def __init__(self, spotCount):
        self.spots = [0] * spotCount 
        self.vehicleMap = {} 

    def parkVehicle(self, vehicle: Vehicle) -> bool:
        size = vehicle.getSpotSize() 

        # Sliding window algorithm 
        l, r = 0, 0 
        while r < len(self.spots):
            if self.spots[r] != 0: 
                l = r + 1 
            
            if r - l + 1 == size: 
                # We found enough spots, park the vehicle 
                for i in range(l, r + 1): 
                    self.spots[i] =  1 
                    self.vehicleMap[vehicle] = [l, r]
                    return True 
            
            r += 1 
        return False 

    def removeVehicle(self, vehicle): 
        start, end = self.vehicleMap[vehicle] 
        for i in range(start, end + 1):
            self.spots[i] = 0 

        del self.vehicleMap[vehicle] 

    def getParkingSpots(self):
        return self.spots

    def getVehicleSpots(self, vehicle):
        if vehicle in self.vehicleMap:
            return self.vehicleMap[vehicle]

        return None

In [50]:
class ParkingGarage: 
    def __init__(self, floorCount, spotsPerFloor):
        self.parkingFloors = [] 
        for i in range(0, floorCount):
            self.parkingFloors.append(ParkingFloor(spotsPerFloor))

    def parkVehicle(self, vehicle): 
        for i in range(0, len(self.parkingFloors)):
            parkingFloor = self.parkingFloors[i] 
            if parkingFloor.parkVehicle(vehicle):
                return True 

        return False

    def removeVehicle(self, vehicle):
        for i in range(0, len(self.parkingFloors)):
            parkingFloor = self.parkingFloors[i] 
            if parkingFloor.getVehicleSpots(vehicle) is not None:
                parkingFloor.removeVehicle(vehicle)

                return True 

        return False
                

In [51]:
import datetime 
import math 

class ParkingSystem: 
    def __init__(self, parkingGarage, hourlyRate):
        self.parkingGarage = parkingGarage 
        self.hourlyRate = hourlyRate 
        self.timeParked = {} # Map driver ID to the time they parked 

    def parkVehicle(self, driver):
        currentHour = datetime.datetime.now().hour 
        isParked = self.parkingGarage.parkVehicle(driver.getVehicle())

        if isParked: 
            self.timeParked[driver.getId()] = currentHour

        return isParked

    def removeVehicle(self, driver):
        if driver.getId() not in self.timeParked:
            return False 

        currentHour = datetime.datetime.now().hour
        timeParked = math.ceil(currentHour - self.timeParked[driver.getId()])
        driver.charge(timeParked * self.hourlyRate)

        del self.timeParked[driver.getId()]
        return self.parkingGarage.removeVehicle(driver.getVehicle())

In [55]:
parkingGarage = ParkingGarage(3, 2)
parkingSystem = ParkingSystem(parkingGarage, 5)

driver1 = Driver(1, Car())
driver2 = Driver(2, Limo())
driver3 = Driver(3, Truck()) 

print(parkingSystem.parkVehicle(driver1))      # true
print(parkingSystem.parkVehicle(driver2))      # true
print(parkingSystem.parkVehicle(driver3))      # false

print(parkingSystem.removeVehicle(driver1))    # true
print(parkingSystem.removeVehicle(driver2))    # true
print(parkingSystem.removeVehicle(driver3))    # false

True
True
False
True
True
False
